In [1]:
import sys
import os

current_dir = os.path.dirname(os.path.abspath(''))
others_path = os.path.join(current_dir, '..', 'gpr')

others_path = os.path.abspath(others_path)
if others_path not in sys.path:
    sys.path.append(others_path)

import numpy as np
import matplotlib.pyplot as plt
from collections.abc import Callable
import cvxpy as cp

import eos
from kernels import Kernel
import gaussianprocess
import finitedimensionalgp
import sampling as sam
import prepare_ceft as pc
import prepare_pqcd as pp
import anal_helpers as anal
from pqcd.pQCD import pQCD
from constants import get_phi, ns
import virtobs as vo

from pathlib import Path
notebook_dir = Path.cwd()

from scipy.linalg import cholesky, solve_triangular, cho_solve
from scipy.stats import norm
import scipy as sp
import pandas as pd

# the data

In [2]:
grid_size = 25

import numpy as np
import matplotlib.pyplot as plt

n_pqcd_grid = np.linspace(25, 40, grid_size) * 0.16   # fm⁻³
X_grid = np.geomspace(0.5, 2.0, 40)              # or use np.geomspace

cs2_family = []
for X in X_grid:
    n_raw, cs2_raw = pp.get_pqcd(X)
    n_raw = n_raw * 0.16                        # convert to fm⁻³
    cs2_interp = np.interp(n_pqcd_grid, n_raw, cs2_raw)
    cs2_family.append(cs2_interp)

cs2_family = np.array(cs2_family)               # shape: (n_X, n_n)
cs2_min = cs2_family.min(axis=0)
cs2_max = cs2_family.max(axis=0)

mu_pqcd_grid = np.linspace(2.2,3,grid_size)*1000

mu_family = []
for X in X_grid:
    n_raw, cs2_raw = pp.get_pqcd(X, size=grid_size)
    n_raw = n_raw * 0.16                        # convert to fm⁻³
    mu_interp = np.interp(n_pqcd_grid, n_raw, mu_pqcd_grid)
    mu_family.append(mu_interp)

mu_family = np.array(mu_family)               # shape: (n_X, n_n)
mu_min = mu_family.min(axis=0)
mu_max = mu_family.max(axis=0)



In [3]:
n_ceft, cs2_ceft, cs2_l, cs2_u = anal.get_ceft_cs2()
cs2_ceft_sigma = pc.CI_to_sigma(cs2_u-cs2_l)
cs2_hat, X_hat, sigma_hat, l_hat, alpha_hat = sam.get_hype_samples('SE')

cs2_ceft = cs2_ceft 

n = n_ceft*ns
cs2 = cs2_ceft
cs2_sigma = cs2_ceft_sigma


# set up

In [6]:
def softplus(x):
    #return np.maximum(x, 0) + np.log1p(np.exp(-np.abs(x)))
    return np.log1p(np.exp(x))

def log_likelihood_approx(zeta, A, b, eta):
    """
    Approximate log likelihood for linear constraints A zeta + b >= 0
    using a scaled sigmoid function.

    zeta: (d,) current sample
    A: (m, d) constraint matrix
    b: (m,) constraint offset
    eta: approximation strength

    Returns: scalar log likelihood
    """
    logits = A @ zeta + b
    return -np.sum(softplus(-eta * logits))

def sample_prior(cov):
    """
    Sample from N(0, cov)
    """
    return np.random.multivariate_normal(mean=np.zeros(cov.shape[0]), cov=cov)

def project_onto_constraints(A, b, z):
    """
    Project vector z onto the feasible set defined by A x + b >= 0
    using CVXPY with CLARABEL solver.
    Solves: min_x 0.5 * ||x - z||² subject to A x + b >= 0
    """
    d = z.shape[0]
    x = cp.Variable(d)

    objective = cp.Minimize(0.5 * cp.sum_squares(x - z))
    constraints = [A @ x + b >= 0]
    problem = cp.Problem(objective, constraints)
    
    problem.solve(solver=cp.CLARABEL, verbose=False)

    if problem.status not in ["optimal", "optimal_inaccurate"]:
        raise RuntimeError(f"Projection failed: {problem.status}")
    
    return x.value

def elliptical_slice_sampling(f, cov, A, b, eta, log_likelihood_fn, max_attempts=100000):
    nu = sample_prior(cov)
    log_y = log_likelihood_fn(f, A, b, eta) + np.log(np.random.uniform())

    theta = np.random.uniform(0, 2 * np.pi)
    theta_min = theta - 2 * np.pi
    theta_max = theta

    for _ in range(max_attempts):
        f_prime = f * np.cos(theta) + nu * np.sin(theta)
        ll = log_likelihood_fn(f_prime, A, b, eta)

        if ll > log_y:
            print(f"θ: {theta:.2f}, log_y: {log_y:.2f}, log_ll: {ll:.2f}, min_constraint: {np.min(A @ f_prime + b):.4f}")

            violation = A @ f_prime + b
            if np.all(violation >= 0):
                return f_prime
            elif np.min(violation) > -1e-6:
                # Mild violation → likely due to approximation → project
                try:
                    projected = project_onto_constraints(A, b, f_prime)
                    return projected
                except RuntimeError:
                    print("Projection failed")
                    continue

            # Else: do not accept — continue slicing
        # Not accepted: shrink bracket
        if theta < 0:
            theta_min = theta
        else:
            theta_max = theta
        theta = np.random.uniform(theta_min, theta_max)

    raise RuntimeError("ESS failed to find valid sample after max_attempts.")

def sample_tmvn_ess(mu, cov, A, b, X, y, n_samples, burn_in=100, eta_init=20.0, update_eta=False):
    d = mu.shape[0]
    zeta = mu.copy()
    samples = []
    eta = eta_init
    count = 0

    while len(samples) < n_samples:
        zeta_centered = zeta - mu
        zeta_centered = elliptical_slice_sampling(
            f=zeta_centered,
            cov=cov,
            A=A,
            b=b,
            eta=eta,
            log_likelihood_fn=lambda z, *_: log_likelihood_approx(z, A, b, eta)
        )
        zeta = zeta_centered + mu
        if np.all(A @ zeta + b >= 0):
            samples.append(zeta.copy())

        if update_eta:
            eta *= 1.01

        if count > n_samples + 10:
            raise ValueError("Max iterations reached")
        count += 1
    samples = np.array(samples)
    samples = samples[burn_in:]
    return samples

def sample_posterior(self, init, n_samples, eta=20, burn_in=1000, update_eta=True):
    mu = init
    cov = np.linalg.inv(self.cov_star_inverse)
    A = self.constraints["M"]
    b = self.constraints["g"]
    X = self.Phi_matrix  # assume you store design matrix in self.X
    y = self.y_train  # assume targets are in self.y

    samples = sample_tmvn_ess(mu, cov, A, b, X, y, n_samples + burn_in, eta_init=eta, update_eta=update_eta)
    return samples[burn_in:]

In [4]:
from virtobs import VOGP